# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk

nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\ivant\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\ivant\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\ivant\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\ivant\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\ivant\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

True

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [3]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
    
for key, value in obj.items():
    print(f'"{key:<{max([len(x) for x in list(obj.keys())])}}" = "{value}"')

"home_page" = "https://github.com/pypa/sampleproject"
"keywords " = "sample setuptools development"
"license  " = "MIT"


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
(obj.str.extract(r"(\d+[-]\d+)", expand=False)).values

array(['19-1', '20-4', '20-3'], dtype=object)

3. Разбейте текст формулировки задачи 2 на слова.

In [5]:
re.findall(r"\w+", 'Написать регулярное выражение,которое позволит найти номера групп студентов.')

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [6]:
recipes = pd.read_csv('recipes_sample.csv')
recipes[recipes['id']==170895]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
15754,leeks and parsnips sauteed or creamed,170895,27,8377,2006-05-31,21.0,this is good sauteed only or creamed. very eas...,9.0


In [7]:
def format_table(recipes):
  header_row = f"|    {'id':^{max(len(str(id)) for id in recipes['id'])}}    |  {'minutes':^{max(len(str(minutes)) for minutes in recipes['minutes'])}}  |\n"
  header_row += f"|{'-' * ((len(header_row))-3)}|\n"
  
  data_rows = ""
  for index, row in recipes.iterrows():
    data_row = f"|{row['id']:^{len(header_row.split('|')[1])}}|{row['minutes']:^{len(header_row.split('|')[2])}}|\n"
    data_rows += data_row

  table = header_row + data_rows
  return table

print(format_table(recipes[['id','minutes']].sample(5)))

|      id      |  minutes  |
|--------------------------|
|    84434     |    30     |
|    84042     |    65     |
|    480161    |    35     |
|    37428     |    25     |
|    324055    |    65     |



2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [8]:
def show_info(recipe_id):
  recipes_data = pd.read_csv("recipes_sample.csv")
  recipe = recipes_data[recipes_data["id"] == recipe_id]

  # Загрузка данных о шагах приготовления из XML-файла
  with open('steps_sample.xml') as f:
    ab = BeautifulSoup(f, 'xml')
  
  res = dict([(int(recipe.find("id").next),[steps.next for steps in recipe.steps.find_all("step")]) for recipe in ab.recipes.find_all("recipe")])

  # Поиск шагов для данного рецепта
  recipe_steps = res[recipe_id]

  # Формирование описания рецепта
  recipe_info = f"\n\"{recipe['name'].iloc[0]}\"\n\n"
  
  step_num = 1
  for step in recipe_steps:
    recipe_info += f"{step_num}. {step.text}\n"
    step_num += 1
    
  recipe_info += "----------\n"
  recipe_info += f"Автор: {recipe['contributor_id'].iloc[0]}\n"
  recipe_info += f"Среднее время приготовления: {recipe['minutes'].iloc[0]} минут\n"

  return recipe_info

print(show_info(170895))


"leeks and parsnips  sauteed or creamed"

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an add

### Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [9]:
def find_time(recipe_id,pattern):
    recipes_data = pd.read_csv("recipes_sample.csv")
    recipe = recipes_data[recipes_data["id"] == recipe_id]

    # Загрузка данных о шагах приготовления из XML-файла
    with open('steps_sample.xml') as f:
        ab = BeautifulSoup(f, 'xml')
    
    res = dict([(int(recipe.find("id").next),np.array([steps for steps in recipe.steps.find_all("step")])[[len(re.findall(pattern,str(steps)))!=0 for steps in recipe.steps.find_all("step")]]) for recipe in ab.recipes.find_all("recipe")])
    

    return res[recipe_id]
pattern = r"(\d+) (?:hour|hours|minute|minutes)"
find_time(25082,pattern)

array([['turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board'],
       ['when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly'],
       ['let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours'],
       ['when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc'],
       ['bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom']],
      dtype='<U182')

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [36]:
pattern = r'\Athis[a-zA-Z0-9_ ]*, ?but'
obj = recipes['description'].str.contains(pattern).replace({np.nan:False})
print(obj.sum())
pd.set_option('display.max_colwidth',None)
recipes[obj][['description']].sample(3)

133


,description
29340,"this is so good, but does require 24 hours marinating time. \r\nthe sauce forms a juicy crust around the meat which remains moist and tender."
24245,"this is a simple, but yet delicious, creamy, baked corn casserole. it was given to my sister and i from a dear friend. we do not know where she got the recipe. we fix this dish quite often around our house."
17916,"this is absolutely delicious, but you have to be a mollasses and spice fan. this can be served as a dessert with either a topping of applesauce,crushed pineapple or cool whip."


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [11]:
def rep_frac(recipe_id):
    recipes_data = pd.read_csv("recipes_sample.csv")
    recipe = recipes_data[recipes_data["id"] == recipe_id]

    # Загрузка данных о шагах приготовления из XML-файла
    with open('steps_sample.xml') as f:
        ab = BeautifulSoup(f, 'xml')
    
    pattern = r'\s*/\s*'
    
    res = dict([(int(recipe.find("id").next),(re.sub(pattern,'/','hihihiha'.join([steps.next for steps in recipe.steps.find_all("step")]))).replace('hihihiha',' \n')) for recipe in ab.recipes.find_all("recipe")])
    

    return res[recipe_id]

print(rep_frac(72367))

mix butter , flour , 1/3 c 
sugar and 1-1/4 t 
vanilla 
press into greased 9" springform pan 
mix cream cheese , 1/4 c 
sugar , eggs and 1/2 t 
vanilla beating until fluffy 
pour over dough 
combine apples , 1/3 c 
sugar and cinnamon 
arrange on top of cream cheese mixture and sprinkle with almonds 
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [12]:
with open('steps_sample.xml') as f:
  ab = BeautifulSoup(f, 'xml')
  res = dict([(int(recipe.find("id").next),[list(map(lambda x: x.upper(),word_tokenize(steps.next))) for steps in recipe.steps.find_all("step")]) for recipe in ab.recipes.find_all("recipe")])
  wordset=[]
  for k,v in res.items():
    res[k]= []
    for i in range(len(v)):
      for j in range(len(v[i])):
        if v[i][j].isalpha():
          res[k].append(v[i][j])
    wordset.extend(res[k])
  wordset=set(wordset)


len(wordset)

14926

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [13]:
longest_sentences = []

for i in recipes.index:
    
    record = recipes.loc[i, :]
    description = record.description
    
    if type(description)==str:
        sentences = sent_tokenize(description)
        
    for sentence in sentences:
        cur_len = len(sentence)
        longest_sentences.append([cur_len, i])
        
        cur_len = len(sentence)
        cur_len = len(sentence)
        
recipes.loc[list(map(lambda l: l[1], sorted(longest_sentences, key=(lambda l: l[0]), reverse=True)[:5])), :]


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
15407,kittencal s cabbage rolls with tomato sauce,150485,220,89831,2006-01-06,NaN,this is my own recipe that i have made for ove...,13.0
15424,kittencal s four cheese lasagna,272423,1480,89831,2007-12-18,31.0,i strongly suggest due to the large amount of ...,21.0
15451,kittencal s scalloped potato and ground beef c...,215414,110,89831,2007-03-07,13.0,sliced potatoes in a delicious creamy sauce on...,NaN
13827,homemade smoky veggie bacon salt,508965,10,37449,2013-11-05,NaN,this stuff is addictive and the uses are endle...,11.0
15423,kittencal s famous coleslaw,102617,1440,89831,2004-10-26,NaN,this is the most delicious coleslaw that you w...,13.0


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [14]:
toktok = nltk.tokenize.toktok.ToktokTokenizer()

def parts_of_speech(s: str):
    words_data = [(token, ll := nltk.pos_tag([token])[0][1], len(token), len(ll)) for token in toktok.tokenize(s)]
    
    first_l = []
    second_l = []
    
    for tupl in words_data:
        ll = max(tupl[2], tupl[3])
        
        first_l.append(f'{tupl[1]:^{ll}}')
        second_l.append(f'{tupl[0]:^{ll}}')
    
    first = ' '.join(first_l)
    second = ' '.join(second_l)
    
    rezult = '\n'.join((first, second))
    
    return rezult

print(parts_of_speech(recipes[recipes['id']==241106]['name'].values[0]))

   NN     NNS    IN     NNS     NN    NN   CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives
